# Complementaria 4

# Ejercicio 1 - Sector eléctrico

Una gran empresa del sector eléctrico cuenta con un equipo que se encuentra sujeto a fallas menores, causadas por eventos como descargas atmosféricas o sobretensiones. Como parte del equipo de analítica de la empresa, se le ha encargado de desarrollar un modelo que permita estimar los costos de mantenimiento de este tipo de equipos, con el fin de contar con criterios de compra sólidos para futuras adquisiciones.

A partir de información histórica se ha estimado que las fallas menores ocurren de acuerdo con un proceso de **Poisson**, con una frecuencia de **10 por mes** (donde un mes se considera que tiene 30 días).

Cuando ocurre una falla menor, se activa una protección y se inicia un procedimiento de corrección que permite resolver la falla. El **tiempo de corrección** se ha estimado que sigue una distribución **exponencial** con **media de 5 minutos**.

Si en un equipo se encuentra activa 1 falla, pueden ocurrir fallas adicionales, hasta un máximo de **10 fallas**. Para cada falla activa la operación de corrección es **independiente** de las demás.

Cuando el equipo acumula 10 fallas, se pierde el procedimiento de corrección, el equipo sale de operación y se debe activar un protocolo de reparación que toma un **tiempo exponencial con media de 1 día**. Cuando se cumple con la reparación, el equipo vuelve a su estado inicial sin fallas.

---

## **a.** Formule una **cadena de Markov** que permita realizar el seguimiento del estado de un equipo con respecto a sus fallas, correcciones y reparaciones. El modelo debe usar **minutos** como unidad de tiempo. Defina explícitamente todos los componentes de su modelo.
### Parámetros
- Las fallas ocurren como un proceso de **Poisson**, tienen una frecuencia de 10 por mes. $\lambda=10\frac{\text{fallas}}{\text{mes}} \rightarrow 0.00023148\text{ min}^{-1}$
- Las fallas se corrigen en tiempo exponencial con $\mu=\frac{1\text{ falla}}{5\text{ min}}=0.2\text{ min}^{-1}$
- Si se acumulan 10 fallas, se arreglan todas con un tiempo expoenncial con media de 1 dia = 1440 minutos = $\gamma=\frac{1}{1440}=0.0006944\text{ min}^{-1}$, ya que no compiten en si es un solo reloj exponencial.
### Variable
$$
X_{n}=\text{Numero de equipos con fallas en el tiempo }t
$$

### Espacio de estado
$$
S_{x}=\{0,1,2,3,4,5,6,7,8,9,10\}
$$
### Matriz de generación

$$
\begin{bmatrix}
&0&1&2&3&4&5&6&7&8&9&10 \\
0 &-\lambda&\lambda&0&0&0&0&0&0&0&0&0\\
1 & 1\mu&-(1\mu+\lambda)&\lambda&0&0&0&0&0&0&0&0\\
2 & 0 &2\mu & -(2\mu+\lambda)&\lambda&0&0&0&0&0&0&0 \\
3 & 0 & 0&3\mu&-(3\mu+\lambda)&\lambda&0&0&0&0&0&0 \\
4 \\
5 \\
6 \\
7 \\ 
8 \\
9 \\
10&\gamma&0&0&0&0&0&0&0&0&0&-\gamma
\end{bmatrix}
$$


$$
P_{i,j}=\begin{cases}
\lambda&j=i+1 \\
n\mu&j=i-1,j > 0,n=i \\
\gamma&j=0 \land i=10 \\
0&d.l.c
\end{cases}
$$


In [1]:
from jmarkov.ctmc import ctmc
from jmarkov.dtmc import dtmc
import numpy as np

### Problema 1

In [3]:
# ----------
# Parametros
# ----------

# Ocurrencia de fallas como proceso de Poisson, convertido de meses a minutos
lmbda = (10/1)*(1/30)*(1/24)*(1/60)

# Fallas se corrigen en tiempo exponencial, en minutos

mu = 1/5

#  Si se acumulan más de 10 fallas, se arreglan todas con gamma
gamma = 1/1440

# Numero maximo de fallas
n = 10

estados = np.array([i for i in range(0,n+1)])

matriz_transicion = np.zeros((len(estados),len(estados)))

for i in range(len(estados)):
    for j in range(len(estados)):
        if j == i +1:
            matriz_transicion[i,j]=lmbda
        elif j == i - 1 and j > 0:
            matriz_transicion[i,j]=i*mu
        elif j == 0 and i == n:
            matriz_transicion[i,j] = gamma
        else:
            matriz_transicion[i,j] = 0
            
    matriz_transicion[i][i] = -np.sum(matriz_transicion[i])
            
matriz_transicion
        


array([[-2.31481481e-04,  2.31481481e-04,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -2.31481481e-04,  2.31481481e-04,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  4.00000000e-01, -4.00231481e-01,
         2.31481481e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  6.00000000e-01,
        -6.00231481e-01,  2.31481481e-04,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         8.00000000e-01, -8.00231481e-01,  2.31481481e-04,
  

In [5]:
cadena = ctmc(matriz_transicion, estados)

# Ejercicio 2 - AutoPark

Usted y su grupo de amigos tienen planeado lanzar una startup, **AutoPark**, que ofrecerá un servicio de parqueadero automático (robotizado) en zonas de la ciudad con limitaciones de espacio. Su sistema inicial tiene una configuración que permite tener hasta **3 vehículos parqueados** al mismo tiempo.

Debido a las restricciones de pico y placa, un día de operación se puede resumir de la siguiente manera:

* Al iniciar el día, cada usuario decide si retira su vehículo o no. Se ha observado que cada usuario retira su vehículo con **probabilidad $r$** independiente de los demás. Es decir, si hay $i$ vehículos parqueados, la probabilidad de que $k$ de los $i$ se vayan del parqueadero corresponde a:

  $$
  b_{i,k} = \binom{i}{k} r^k (1-r)^{\,i-k},
  $$

  siguiendo el comportamiento de una variable aleatoria binomial.

* Durante el día puede llegar un vehículo a utilizar el parqueadero, lo cual ocurre con **probabilidad $p$**. Si llega un vehículo y hay al menos un espacio disponible, éste se queda por lo menos hasta el día siguiente. Si no hay espacios disponibles, el vehículo no ingresa y se considera una **venta perdida**.

---

## **a.** Modele el sistema de AutoPark como una **cadena de Markov** para estudiar su desempeño y así poder evaluar el impacto de algunas decisiones de diseño. Defina claramente todos los componentes del modelo.


**Pista:** Observe el sistema al inicio de cada día y considere el orden en que ocurren los eventos, primero las **salidas** y luego los posibles **arribos**. Considere por ejemplo que si el sistema se encuentra lleno al iniciar el día, algunos vehículos podrían salir, dejando espacio para la llegada de un nuevo vehículo durante el día.

### Parámetros
- Máximo tres carros al tiempo
- El usuario retira su vehiculo con probabilidad $r$ independiente que sigue una distribucion binomial tal que $b_{i,k} = \binom{i}{k} r^k (1-r)^{\,i-k}$
- Un carro llega con probabilidad $p$
- **Primero deben salir y luego si pueden llegar**

### Variable
$$
X_{n}=\text{Número de carros parqueados al inicio del dia }n
$$
### Espacio de Estados
$$
S_{x}=\{0,1,2,3\}
$$
### Matriz de transición

$$
\begin{bmatrix}
&0&1&2&3 \\
0&b_{0,0}*(1-p)&b_{0,0}*p&0&0 \\
1&b_{1,1}*(1-p)&(b_{1,0}*(1-p))+(b_{1,1}*p)&b_{1,0}*p&0 \\
2&b_{2,2}*(1-p)&(b_{2,1}*(1-p))+(b_{2,2}*p)&(b_{2,0}*(1-p))+(b_{2,1}*p)&b_{2,0}*p \\
3&b_{3,3}*(1-p)&(b_{3,2}*(1-p))+(b_{3,3}*p)&(b_{3,1}*(1-p))+(b_{3,2}*p)&(b_{3,0}*(1-p))+(b_{3,1}*p)+(b_{3,0}*p)
\end{bmatrix}
$$

$$
\begin{cases}
b_{i,i}*(1-p)&j = 0 \\
b_{i,0}*p&j=i+1, i\in S_{x}\land j\in S_{x} \\ 
(b_{i,k}*(1-p))+(b_{i,i-j+1}*p)& j=i-k,k=\{0,\dots i\},i>0\land j<3\\
(b_{i,k}*(1-p))+(b_{i,i-j+1}*p)+(b_{i,0}*p)& j=i-k,k=\{0,\dots i\},i=3\land j=3 \\
\end{cases}
$$


In [6]:
from scipy.stats import binom

# Uso: binom.pmf(k, n, r)

In [ ]:
# Parametros
N = 3 # Maximo de carros
r = 0.3 # Retiran carros con esta probabilidad independiente
p = 0.6 # Un carro llega con probabilidad p

estados = np.array([i for i in range(0,N+1)])
matriz_transicion = np.zeros((len(estados),len(estados)))

for i in estados:
    for j in estados:
            
            if j == 0:
                
                print(f"{i},{j} entro a j=0")
                matriz_transicion[i,j] = binom.pmf(i,i,r) * (1-p)
            elif j == i + 1 and i in estados and j in estados:
                print(f"{i},{j} entro a j=i+1")
                matriz_transicion[i,j]=binom.pmf(0,i,r) * p
            for k in range(0, i+1): 
                
                if j == i - k and i > 0 and j < 3 and j != 0:
                    print(f"{i},{j} entro a j=i-k")
                    matriz_transicion[i,j] += (binom.pmf(k,i,r)*(1-p))+(binom.pmf(i-j+1,i,r)*p)
                elif j == i-k and i == 3 and j == 3:
                    print(f"{i},{j} entro a j=i-k con i=3 y j=3")
                    matriz_transicion[i,j] += (binom.pmf(k,i,r)*(1-p))+(binom.pmf(i-j+1,i,r)*p) + (binom.pmf(0,i,r)*p)
        
import pandas as pd

df = pd.DataFrame(np.round(matriz_transicion, 4), index=estados, columns=estados)
print(df)

0,0 entro a j=0
0,1 entro a j=i+1
1,0 entro a j=0
1,1 entro a j=i-k
1,2 entro a j=i+1
2,0 entro a j=0
2,1 entro a j=i-k
2,2 entro a j=i-k
2,3 entro a j=i+1
3,0 entro a j=0
3,1 entro a j=i-k
3,2 entro a j=i-k
3,3 entro a j=i-k con i=3 y j=3
        0       1    2    3
0  0.4000  0.6000  0.0  0.0
1  0.1200  0.1800  0.0  0.0
2  0.0360  0.0540  0.0  0.0
3  0.0108  0.0162  0.0  0.0


In [23]:
cadena = dtmc(matriz_transicion, estados)

ValueError: the rows of transition matrix do not sum 1 or has non positive values